In [1]:
# movielens 100K

#SVD 알아오기
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from pathlib import Path

path = Path('../../../datafile/Recsys/ml-100k')
train = pd.read_csv(path /'u4.base',sep='\t',
                   usecols=[0,1,2],names=['user','item','rating'],
                   dtype={'user':int,'item':int,'rating':float})
test = pd.read_csv(path /'u4.test',sep='\t',
                   usecols=[0,1,2],names=['user','item','rating'],
                   dtype={'user':int,'item':int,'rating':float})

R_train = pd.pivot_table(train,values='rating',index='user',columns='item')

In [2]:
R_train.mean().mean()

np.float64(3.078817254018804)

In [2]:
import matfac
mf_model = matfac.MatrixFactorization(k=10,lr=0.0001,reg_param=0.1,epochs=10)
R_train = R_train.fillna(0)
 
print('Start Model Training')
mf_model.fit(R_train)
print('\nModel Training Success')

#Prediction
prediction,P,Q,test_filtered = mf_model.predict(test)    

Start Model Training
Epoch : 0 , RMSE : 3.1480

Model Training Success


In [66]:
P_df = pd.DataFrame(P,index=R_train.index)
Q_df = pd.DataFrame(Q,index=R_train.columns)

In [ ]:
uilist_train = list(zip(train['user'],train['item']))
test_filtered = test[test['user'].isin(uilist_train[0]) | test['item'].isin(uilist_train[1])]
test_filtered
uilist_test = list(zip(test_filtered['user'],test_filtered['item']))

In [ ]:
uilist_test = list(zip(test['user'],test['item']))
pred = []
for val in uilist_test:
    pred.append(np.dot(P_df.loc[val[0]],Q_df.loc[val[1]].T))
test['prediction'] = pred  
prediction = test.copy()
prediction['rating'] = pred

In [4]:
from sklearn.metrics import root_mean_squared_error
final_rmse = root_mean_squared_error(prediction['rating'].values,test_filtered['rating'].values)
print(f'RMSE: {final_rmse:.4f}')

RMSE: 4.8693


In [36]:
c = test.loc[(test['user']==1) & (test['rating']>=0.3)].sort_values('rating',ascending=False)[:10]
d = prediction.loc[(test['user']==1) & (test['rating']>=0.3)].sort_values('rating',ascending=False)[:10]
hit = set(c['item']).intersection(set(d['item']))
len(hit) / 10

0.0

In [27]:
a= test.loc[test['user']==1].sort_values('rating',ascending=False)[:30]
b= test.loc[test['user']==1].sort_values('prediction',ascending=False)[:30]
hit = set(a['item']).intersection(set(b['item']))
hit

{12, 60, 100}

In [43]:
test.sort_values(['user','rating']).reset_index(drop=True)

,user,item,rating,prediction
0,1,74,1.0,-4.636218
1,1,78,1.0,-4.293500
2,1,103,1.0,3.207766
3,1,104,1.0,6.743181
4,1,112,1.0,-3.364657
...,...,...,...,...
19995,458,648,4.0,0.874911
19996,458,1101,4.0,0.606245
19997,459,934,3.0,5.956687
19998,460,10,3.0,-2.650224


In [5]:
uilist_test = list(zip(test['user'],test['item']))
pred_list = []
for ind in uilist_test:
    pred_list.append(R_train[ind[0],ind[1]])
pred_list

KeyError: (877, 381)

In [ ]:
user_item_mat = pd.pivot_table(data=raw_data,values='rating',columns='user',index='item')
user_item_mat

user,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
item,,,,,,,,,,,,,,,,,,,,,
1,5.0,4.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,4.0,...,2.0,3.0,4.0,NaN,4.0,NaN,NaN,5.0,NaN,NaN
2,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,...,5.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
5,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
import numpy as np

# 분해할 행렬 A 생성
A = np.array([
    [3, 1, 1],
    [-1, 3, 1]
])

print("--- 원본 행렬 A ---")
print(A)

U, s, Vt = np.linalg.svd(A)

print("\n--- 분해된 행렬들 ---")
print("U의 모양:", U.shape)
print("s의 모양:", s.shape) # 1차원 배열임에 주의!
print("Vt의 모양:", Vt.shape)

# s를 대각 행렬 Σ로 재구성
# 원본 A의 크기에 맞게 0을 채워줌
Sigma = np.zeros(A.shape)
Sigma[:len(s), :len(s)] = np.diag(s)

k = 2
U_k = U[:,:k]
Sigma_k = Sigma[:k,:k]
Vt_k = Vt[:k,:]

print("\n--- 재구성된 대각 행렬 Σ ---")
print(Sigma)

# U, Σ, Vt를 다시 곱해서 A를 복원
A_restored = U @ Sigma @ Vt  # @는 행렬 곱셈 연산자
A_k_restored = U_k @ Sigma_k @ Vt_k

print("\n--- 복원된 행렬 A,A_k ---")
print('Truncated SVD: ',A_k_restored.round(3))
print('Full-SVD : ',A_restored.round(3)) # 부동소수점 오차를 감안해 반올림

--- 원본 행렬 A ---
[[ 3  1  1]
 [-1  3  1]]

--- 분해된 행렬들 ---
U의 모양: (2, 2)
s의 모양: (2,)
Vt의 모양: (3, 3)

--- 재구성된 대각 행렬 Σ ---
[[3.46410162 0.         0.        ]
 [0.         3.16227766 0.        ]]

--- 복원된 행렬 A,A_k ---
Truncated SVD:  [[ 3.  1.  1.]
 [-1.  3.  1.]]
Full-SVD :  [[ 3.  1.  1.]
 [-1.  3.  1.]]
